In [ ]:
import os
import json
import glob
import requests
import zipfile
import base64
import io
import pandas as pd
import numpy as np
from urllib.parse import urlencode
from parsons import S3, Redshift, Table

config_vars = {
    # LegiScan
    "LEGISCAN_API_KEY": "",
    # S3
    "AWS_ACCESS_KEY_ID": "",
    "AWS_SECRET_ACCESS_KEY": "",
    "BUCKET": "",
    # Civis
    "CIVIS_DATABASE": "",
    "CIVIS_API_KEY": "",
    # Redshift
    "REDSHIFT_PORT": "",
    "REDSHIFT_DB": "",
    "REDSHIFT_HOST": "",
    "REDSHIFT_CREDENTIAL_USERNAME": "",
    "REDSHIFT_CREDENTIAL_PASSWORD": "",
    "S3_TEMP_BUCKET": ""}

for name, value in config_vars.items():
    if value.strip() != "":
        os.environ[name] = value

rs = Redshift()
table = Table()
s3 = S3()

In [ ]:
# pylegiscan functions adapted from https://github.com/poliquin/pylegiscan/blob/master/pylegiscan/legiscan.py

# current aggregate status of bill
BILL_STATUS = {1: "Introduced",
               2: "Engrossed",
               3: "Enrolled",
               4: "Passed",
               5: "Vetoed",
               6: "Failed/Dead"}

# significant steps in bill progress.
BILL_PROGRESS = {1: "Introduced",
                 2: "Engrossed",
                 3: "Enrolled",
                 4: "Passed",
                 5: "Vetoed",
                 6: "Failed/Dead",
                 7: "Veto Override",
                 8: "Chapter/Act/Statute",
                 9: "Committee Referral",
                10: "Committee Report Pass",
                11: "Committee Report DNP"}

# valid legiscan state abbreviations
STATES = ['ak', 'al', 'ar', 'az', 'ca', 'co', 'ct', 'dc', 'de', 'fl', 'ga',
          'hi', 'ia', 'id', 'il', 'in', 'ks', 'ky', 'la', 'ma', 'md', 'me',
          'mi', 'mn', 'mo', 'ms', 'mt', 'nc', 'nd', 'ne', 'nh', 'nj', 'nm',
          'nv', 'ny', 'oh', 'ok', 'or', 'pa', 'ri', 'sc', 'sd', 'tn', 'tx',
          'ut', 'va', 'vt', 'wa', 'wi', 'wv', 'wy']

class LegiScanError(Exception):
    pass

class LegiScan(object):
    BASE_URL = 'http://api.legiscan.com/?key={0}&op={1}&{2}'

    def __init__(self, apikey=None):
        """LegiScan API.  State parameters should always be passed as
           USPS abbreviations.  Bill numbers and abbreviations are case
           insensitive.  Register for API at http://legiscan.com/legiscan
        """
        # see if API key available as environment variable
        if apikey is None:
            apikey = os.getenv('LEGISCAN_API_KEY')
        self.key = apikey.strip()

    def _url(self, operation, params=None):
        """Build a URL for querying the API."""
        if not isinstance(params, str) and params is not None:
            params = urlencode(params)
        elif params is None:
            params = ''
        return self.BASE_URL.format(self.key, operation, params)

    def _get(self, url):
        """Get and parse JSON from API for a url."""
        req = requests.get(url)
        if not req.ok:
            raise LegiScanError('Request returned {0}: {1}'\
                    .format(req.status_code, url))
        data = json.loads(req.content)
        if data['status'] == "ERROR":
            raise LegiScanError(data['alert']['message'])
        return data

    def get_session_list(self, state):
        """Get list of available sessions for a state."""
        url = self._url('getSessionList', {'state': state})
        data = self._get(url)
        return data['sessions']

    def get_dataset_list(self, state=None, year=None):
        """Get list of available datasets, with optional state and year filtering.
        """
        if state is not None:
            url = self._url('getDatasetList', {'state': state})
        elif year is not None:
            url = self._url('getDatasetList', {'year': year})
        else:
            url = self._url('getDatasetList')
        data = self._get(url)
        return data['datasetlist']

    def get_dataset(self, id, access_key):
        """Get list of available datasets, with optional state and year filtering.
        """
        url = self._url('getDataset', {'id': id, 'access_key': access_key})
        data = self._get(url)
        return data['dataset']
    
    def get_session_people(self, session_id=None):
        """Get list of bills for the current session in a state or for
           a given session identifier.
        """
        if session_id is not None:
            url = self._url('getSessionPeople', {'id': session_id})
        else:
            raise ValueError('Must specify session identifier.')
        data = self._get(url)
        return data['sessionpeople']
      
    def get_master_list(self, state=None, session_id=None):
        """Get list of bills for the current session in a state or for
           a given session identifier.
        """
        if state is not None:
            url = self._url('getMasterList', {'state': state})
        elif session_id is not None:
            url = self._url('getMasterList', {'id': session_id})
        else:
            raise ValueError('Must specify session identifier or state.')
        data = self._get(url)
        return [data['masterlist'][i] for i in data['masterlist']]
    
    def get_master_list_raw(self, state=None, session_id=None):
        """Get list of bills for the current session in a state or for
           a given session identifier, optimized for change hash detection.
        """
        if state is not None:
            url = self._url('getMasterListRaw', {'state': state})
        elif session_id is not None:
            url = self._url('getMasterListRaw', {'id': session_id})
        else:
            raise ValueError('Must specify session identifier or state.')
        data = self._get(url)
        return [data['masterlist'][i] for i in data['masterlist']]

    def get_bill(self, bill_id=None, state=None, bill_number=None):
        """Get primary bill detail information including sponsors, committee
           references, full history, bill text, and roll call information.

           This function expects either a bill identifier or a state and bill
           number combination.  The bill identifier is preferred, and required
           for fetching bills from prior sessions.
        """
        if bill_id is not None:
            url = self._url('getBill', {'id': bill_id})
        elif state is not None and bill_number is not None:
            url = self._url('getBill', {'state': state, 'bill': bill_number})
        else:
            raise ValueError('Must specify bill_id or state and bill_number.')
        return self._get(url)['bill']

    def get_roll_call(self, roll_call_id):
        """Roll call detail for individual votes and summary information."""
        data = self._get(self._url('getRollcall', {'id': roll_call_id}))
        return data['roll_call']

legis = LegiScan()

In [ ]:
""" If datasets is not defined in the namespace, this cell will create one API call """

statecode = 'me'

if 'datasets' not in locals():
    datasets = legis.get_dataset_list(state = statecode)

if 'current_session_id' not in locals():
    current_session_id = datasets[0]['session_id']

In [ ]:
""" Running this cell will create one API call """

""" Orignal bulk import: use session_id and session_name to extract all files
    for each dataset."""

for dataset in datasets:
    session_id = dataset['session_id']
    access_key = dataset['access_key']
    details = legis.get_dataset(session_id, access_key)
    z_bytes = base64.b64decode(details['zip'])
    z = zipfile.ZipFile(io.BytesIO(z_bytes))
    z.extractall("./legiscan_api_data")

In [ ]:
""" Make lists of session_id and session_name to use as lookups when adding
    session_id to people jsons"""

session_id_list = []
session_name_list = []

for dataset in datasets:
    session_id = dataset['session_id']
    session_name = dataset['session_name'].replace(' ','_')
    session_id_list.append(session_id)
    session_name_list.append(session_name)

In [ ]:
""" Gather individual JSON files into single list for each file type (bills,
    roll-calls, people). Append session_id to the person records."""

bill_files = glob.glob("legiscan_api_data/*/*/bill/*.json")
people_files = glob.glob("legiscan_api_data/*/*/people/*.json")
vote_files = glob.glob("legiscan_api_data/*/*/vote/*.json")


vote_json = []

for file in vote_files:
    with open(file, "rb") as input_file:
        data = json.load(input_file)['roll_call']
        vote_json.append(data)
        
vote_df = pd.json_normalize(vote_json)
        
    
# remove session from bill (contains duplicative columns)       
bill_json = []

for file in bill_files:
    with open(file, "rb") as input_file:
        data = json.load(input_file)['bill']
        bill_json.append(data)
        
bill_df = pd.json_normalize(bill_json)

        
# for people, must add session_id to each json file        
people_json = []

for file in people_files:
    with open(file, "rb") as input_file:
        session = file.split('/')[2]
        name = session[10:]
        index = session_name_list.index(name)
        session_id = session_id_list[index]
        data = json.load(input_file)['person']
        data['session_id'] = session_id
        people_json.append(data)
    
people_df = pd.json_normalize(people_json)

In [ ]:
""" Make separate dfs of some nested columns for easier querying, smaller tables"""

# roll call

roll_call_tbl = table.from_dataframe(bill_df).long_table('bill_id','votes')
roll_call_tbl.rename_column('votes_roll_call_id', 'roll_call_id')


# subjects

subjects_tbl = table.from_dataframe(bill_df).long_table('bill_id','subjects')
subjects_tbl.rename_column('subjects_subject_name', 'subject_name')
subjects_tbl.rename_column('subjects_subject_id', 'subject_id');


# bill

df = bill_df

nested_cols = []
for col in df.columns:
    if '.' in col:
        nested_cols.append(col)
    elif isinstance(df[col][0],list):
        nested_cols.append(col)
            
df = df.drop(columns = nested_cols, axis = 1)
             
df.mask(df.applymap(type).eq(list) & ~df.astype(bool), inplace = True)
df.replace({np.nan: None}, inplace = True)
    
bill_df = df

bill_tbl = table.from_dataframe(bill_df)


# vote

vote_tbl = table.from_dataframe(vote_df).long_table(['roll_call_id','bill_id'],'votes')
vote_tbl.rename_column('votes_people_id', 'people_id')


# people

df = people_df

nested_cols = []
for col in df.columns:
    if '.' in col:
        nested_cols.append(col)
    elif isinstance(df[col][0],list):
        nested_cols.append(col)
            
df = df.drop(columns = nested_cols, axis = 1)
             
df.mask(df.applymap(type).eq(list) & ~df.astype(bool), inplace = True)
df.replace({np.nan: None}, inplace = True)
    
people_df = df

people_tbl = table.from_dataframe(people_df)

In [ ]:
""" This cell will completely overwrite all legiscan tables in redshift.
    Use this when loading data for the first time, or if redshift tables 
    become corrupted and need to be replaced from stored JSON files."""

legiscan_tables = [
    {
        'table':'legiscan_api_people',
        'schema': 'lkesich',
        'sortkey': 'people_id',
        'distkey': 'people_id',
        'tbl': people_tbl
    },
    {
        'table':'legiscan_api_roll_calls',
        'schema': 'lkesich',
        'sortkey': 'votes_date',
        'distkey': 'roll_call_id',
        'tbl': roll_call_tbl
    },
    {
        'table':'legiscan_api_bills',
        'schema': 'lkesich',
        'sortkey': 'session_id',
        'distkey': 'bill_id',
        'tbl': bill_tbl
    },
    {
        'table':'legiscan_api_votes',
        'schema': 'lkesich',
        'sortkey': 'roll_call_id, people_id',
        'distkey': 'roll_call_id',
        'tbl': vote_tbl
    },
    {
        'table':'legiscan_api_subjects',
        'schema': 'lkesich',
        'sortkey': 'subject_id',
        'distkey': 'bill_id',
        'tbl': subjects_tbl
    }
]

""" Send to redshift or Civis. If you have not defined a distkey or sortkey, 
    you will get a warning that can be safely ignored."""

for key in legiscan_tables:
    destination = key['schema'] + '.' + key['table']
    tbl = key['tbl']
    dist = key['distkey']
    sort = key['sortkey']
    tbl.to_redshift(
        destination, 
        distkey = dist, 
        sortkey = sort, 
        if_exists = 'drop')

In [ ]:
""" Running this cell will create one API call """

""" Get bill_ids and change_hashes for comparison with legiscan"""

# change hashes from redshift for bills in current session
redshift_hashes = rs.query(f'''
    select
        bill_id,
        change_hash
    from lkesich.legiscan_api_bills
    where session_id = {current_session_id}''').to_dataframe()

# roll-calls from redshift for bills in current session
redshift_roll_calls = rs.query(f'''
    select
        roll_call_id
    from lkesich.legiscan_api_roll_calls
        left join lkesich.legiscan_api_bills using(bill_id)
    where session_id = {current_session_id}''').to_dataframe()

redshift_roll_calls = redshift_roll_calls['roll_call_id'].values.tolist()

# change hashes from legiscan for bills in current session
master_list_raw = legis.get_master_list_raw(session_id = current_session_id)
legiscan_hashes = pd.json_normalize(master_list_raw[1:])
test = pd.json_normalize(master_list_raw[1:3])

l = legiscan_hashes
r = redshift_hashes

legiscan_updated = l[l.change_hash.isin(r.change_hash) == False]
legiscan_updated = pd.DataFrame(legiscan_updated).reset_index(drop = True)

In [ ]:
def process_bill(bill):
    """ Take single bill and return Parsons table of bill data formatted to 
        match tables in data warehouse."""
    df = pd.json_normalize(bill)
    
    nested_cols = []
    for col in df.columns:
        if '.' in col:
            nested_cols.append(col)
        elif isinstance(df[col][0],list):
            nested_cols.append(col)
            
    df = df.drop(columns = nested_cols, axis = 1)
             
    df.mask(df.applymap(type).eq(list) & ~df.astype(bool), inplace = True)
    df.replace({np.nan: None}, inplace = True)
    
    bill_tbl = table.from_dataframe(df)
    
    return(bill_tbl)


def process_roll_call(bill):
    """ Take single bill and return Parsons table of roll call data formatted to 
        match tables in data warehouse."""
    df = pd.json_normalize(bill)
             
    df.mask(df.applymap(type).eq(list) & ~df.astype(bool), inplace = True)
    df.replace({np.nan: None}, inplace = True)
    
    roll_call_tbl = table.from_dataframe(df).long_table('bill_id','votes')
    
    if roll_call_tbl.num_rows > 0:
        roll_call_tbl.rename_column('votes_roll_call_id', 'roll_call_id')
    
    return(roll_call_tbl)

    
def process_subjects(bill):
    """ Take single bill and return Parsons table of subject data formatted to 
        match tables in data warehouse."""
    df = pd.json_normalize(bill)
             
    df.mask(df.applymap(type).eq(list) & ~df.astype(bool), inplace = True)
    df.replace({np.nan: None}, inplace = True)
    
    subjects_tbl = table.from_dataframe(df).long_table('bill_id','subjects')
    
    if subjects_tbl.num_rows > 0:
        subjects_tbl.rename_column('subjects_subject_name', 'subject_name')
        subjects_tbl.rename_column('subjects_subject_id', 'subject_id')
    
    return(subjects_tbl)


def process_vote(bill):
    """ Using this function will create one API call per roll-call associated
        with the input bill."""
    
    """ Take single bill and return Parsons table of vote data formatted to 
        match tables in data warehouse."""
    rc_tbl = process_roll_call(bill)
    
    if rc_tbl.num_rows > 0:
        rc_ids = rc_tbl['roll_call_id']
        
        roll_calls = []
        
        for rc_id in rc_ids:
            roll_call = legis.get_roll_call(rc_id)
            roll_calls.append(roll_call)
            
            df = pd.json_normalize(roll_calls)
        
            df.mask(df.applymap(type).eq(list) & ~df.astype(bool), inplace = True)
            df.replace({np.nan: None}, inplace = True)
        
            vote_tbl = table.from_dataframe(df).long_table(['roll_call_id','bill_id'],'votes')
            vote_tbl.rename_column('votes_people_id', 'people_id')
        
            return(vote_tbl)
    

In [ ]:
""" Running this cell will create an API call for each out-of-date bill 
    and an additional API call for each associated roll_call. This may
    result in hundreds or thousands of calls, depending on when the bill 
    table was last updated."""

""" Compare stored hashes with legiscan hashes. For hashes that don't
    match, get the updated bill from the LegiScan API. """

""" This takes a very long time--actually quicker to just rewrite the whole 
    database. Needs to be modified. """


for row in range(len(legiscan_updated)):
    
    bill_id = legiscan_updated['bill_id'][row]
    change_hash = legiscan_updated['change_hash'][row]
    
    rs.query(f'''
        delete from lkesich.legiscan_api_bills
        where
            bill_id = {bill_id}
            and change_hash <> '{change_hash}';''')
    
    rs.query(f'''
        delete from lkesich.legiscan_api_subjects
        where
            bill_id in (
                select
                    bill_id
                from lkesich.legiscan_api_bills
                where
                    bill_id = {bill_id}
                    and change_hash <> '{change_hash}')''')
    
    rs.query(f'''
        delete from lkesich.legiscan_api_roll_calls
        where
            bill_id in (
                select
                    bill_id
                from lkesich.legiscan_api_bills
                where
                    bill_id = {bill_id}
                    and change_hash <> '{change_hash}')''')
    
    rs.query(f'''
        delete from lkesich.legiscan_api_votes
        where
            bill_id in (
                select
                    bill_id
                from lkesich.legiscan_api_bills
                where
                    bill_id = {bill_id}
                    and change_hash <> '{change_hash}')''')
    
    bill = legis.get_bill(bill_id)
    
    bill_tbl = process_bill(bill)
    roll_call_tbl = process_roll_call(bill)
    subjects_tbl = process_subjects(bill)
    vote_tbl = process_vote(bill)
    
    if bill_tbl.num_rows > 0:
        bill_tbl.to_redshift('lkesich.legiscan_api_bills', if_exists = 'append')
        
    if roll_call_tbl.num_rows > 0:
        roll_call_tbl.to_redshift('lkesich.legiscan_api_roll_calls', if_exists = 'append')
        
    if subjects_tbl.num_rows > 0:
        subjects_tbl.to_redshift('lkesich.legiscan_api_subjects', if_exists = 'append')
        
    if not vote_tbl is None:
        if vote_tbl.num_rows > 0:
            vote_tbl.to_redshift('lkesich.legiscan_api_votes', if_exists = 'append')
    